In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split
from feature_engineering import DataProcessor
from LSTM import LSTMModel
from RNN import RNNModel
from trainRNN import ModelTrainer
import torch.nn as nn



# Hyperparameters
input_size = 100
hidden_size = 256
num_layers = 1
batch_size = 64
num_epochs = 10
learning_rate = 0.001

# Load and preprocess data
dataset_dir = ''
data_processor = DataProcessor(dataset_dir)

# Choose to preprocess data or load saved mappings
preprocess_data = False
char_to_id_file = 'char_to_id.json'
id_to_char_file = 'id_to_char.json'

if preprocess_data:
    ids, char_to_id, id_to_char = data_processor.preprocess()
    data_processor.save_mappings(char_to_id_file, char_to_id, id_to_char_file, id_to_char)
else:
    char_to_id, id_to_char = data_processor.load_mappings(char_to_id_file, id_to_char_file)
print('Mapping loaded')
## Create Dataset sequences with pytorch
#data_processor = DataProcessor(dataset_dir)
dialogue_lines = data_processor.read_dialogue_lines()
text = ' '.join(dialogue_lines.values())
ids = data_processor.text_to_ids(text, char_to_id)
print(len(ids))
dataset = data_processor.create_dataset(ids[:1000050])



Mapping loaded
2623471


In [ ]:
len(dataset)

1000000

In [ ]:
from trainRNN import ModelTrainer

train_loader, val_loader, test_loader=DataProcessor.create_loaders(dataset, 0.8, 0.1, 0.1, 50)
print('Data Loaded')
## Train and Evaluate the model
# Initialize the RNN model
input_size = embedding_size = 50
hidden_size = 256
num_layers = 2
vocab_size = len(char_to_id)
model = RNNModel(input_size, hidden_size, vocab_size, num_layers)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
model.to(device)

# Define the loss function, learning rate, and optimizer
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
trainer=ModelTrainer(model, train_loader, criterion, optimizer, device)

print('start training')
# Training loop

num_epochs = 10
for epoch in range(num_epochs):
    print(epoch)
    loss = trainer.train()
    # Evaluate the model on the validation set
    validation_loss = trainer.evaluate(val_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss:.4f}, Validation Loss: {validation_loss:.4f}")

# Evaluation on test set
test_loss = trainer.evaluate(test_loader)


Data Loaded
start training
0
Epoch 1/10, Loss: 1.3664, Validation Loss: 1.3453
1
Epoch 2/10, Loss: 1.2337, Validation Loss: 1.3267
2
Epoch 3/10, Loss: 1.2109, Validation Loss: 1.3207
3
Epoch 4/10, Loss: 1.2016, Validation Loss: 1.3184
4
Epoch 5/10, Loss: 1.1973, Validation Loss: 1.3149
5
Epoch 6/10, Loss: 1.1955, Validation Loss: 1.3156
6
Epoch 7/10, Loss: 1.1946, Validation Loss: 1.3147
7
Epoch 8/10, Loss: 1.1940, Validation Loss: 1.3168
8
Epoch 9/10, Loss: 1.1941, Validation Loss: 1.3160
9
Epoch 10/10, Loss: 1.1943, Validation Loss: 1.3183


In [ ]:
print('Loss on testing data:', test_loss)

Loss on testing data: 1.318735296010971


In [ ]:
# Save model
torch.save(trainer.model, 'rnn_model')

In [ ]:
model=trainer.model


In [ ]:
from evaluateRNN import Evaluater
# Evaluate performance
evaluater = Evaluater(model, device)
perplexity= evaluater.calculate_perplexity(test_loader, criterion)
print('Perplexity:' , perplexity)

Perplexity: 3.7386900498074134


In [ ]:
seed_text = "Start a discussion about coffee"
gen_length = 2000
temperature = 0.8

generated_text = evaluater.generate_text(seed_text, gen_length, char_to_id, id_to_char, device, temperature)
#generated_text = generate_text(model, seed_text, gen_length, char_to_id, id_to_char, device, temperature)
print(generated_text)


Start a discussion about coffee. The sun ever supposed to know it, that is, Madame.  You have no problem. There must be mad. Better than to find out about it. You have a few linetion? I wanted your dreams in the groovy. I'd shot to believe you and yours for what you're doing, Sarah Lawrence you pay our different man. I bet. Well... no. I am sorry, Milo? A why you're true to a helluva girls and the Indies war with his meetended.  And me are you doing?!  He's bleing Daphool. You would have head going in the local place you down to fix cass you say it anything out a man wonderful! Okay. May not at lash, don't believe you.  If I took the dirty safe...  But it's ones to go out was the boached. Good". Well, what? No connection my damage private you guys six, Curke.  It's about to be as a nicked by any patives up. If they were so sorry. You mean I'mmore when you go holding Promancesshand close to the only think about Loodon Airvolves sir. You have you came out of your bachelor party well. The

In [ ]:
import torch.nn.functional as F

def generate_text(model, seed_text, gen_length, char_to_id, id_to_char, device, temperature=1.0):
        model.eval()

        # Convert seed_text to tensor
        input_seq = [char_to_id[char] for char in seed_text]
        input_seq = torch.tensor(input_seq, dtype=torch.long).unsqueeze(0).to(device)  # Add batch_first dimension

        # Initialize the hidden state
        hidden = model.init_hidden(1)

        # Generate text
        generated_text = seed_text
        for _ in range(gen_length):
            with torch.no_grad():
                outputs, hidden = model(input_seq, hidden)
                char_probs = F.softmax(outputs[-1, :] / temperature, dim=0)

                # Sample a character from the output probabilities
                char_idx = torch.multinomial(char_probs, 1).item()

                # Append the generated character to the generated text
                generated_char = id_to_char[str(char_idx)]
                generated_text += generated_char

                # Update the input sequence with the generated character
                input_seq = torch.tensor([[char_idx]], dtype=torch.long).to(device)

        return generated_text